In [6]:
# imports
from subprocess import check_output
import pandas as pd
import plotly.express as px
import plotly
import re
import nbformat
import time
import json

In [9]:
# measure probability calculation efficiency
results_df = pd.DataFrame(columns = ['Processed Cells', 'Retries', 'Time', 'Examinations', 'Movements', 'Actions'])
agents = ["Agent 6", "Agent 7", "Agent 8"]

for x in range(1):
  command = 'python main.py -d 25 -p 0.3'
  out = json.loads(str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 3/src', shell=True)).lstrip("b'").rstrip("'").replace("\\n", ""))

  for i,a in enumerate(agents):
    results_df = results_df.append([{'Agent': i+6, 'Processed Cells': out[a]["processed"], 'Retries': out[a]["retries"], 'Time': out[a]["time"], 'Examinations': out[a]["examinations"], 'Movements': out[a]["movements"], 'Actions': out[a]["examinations"] + out[a]["movements"]}], ignore_index=True)

results_df.head()

,Processed Cells,Retries,Time,Examinations,Movements,Actions,Agent
0,54157,2078,4.089057,1849,13439,15288,6.0
1,66643,2638,4.139534,2289,18201,20490,7.0
2,30513,1119,3.798985,1303,4014,5317,8.0
